In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Data...

In [ ]:
import pandas as pd
import numpy as np
credits=pd.read_csv(r'/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies=pd.read_csv(r'/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
credits.head()

In [ ]:
movies.head()

In [ ]:
credits.rename(columns={'movie_id':'id'},inplace=True)
credits.head()

In [ ]:
movies_df=movies.merge(credits,on='id')
movies_df.head()

In [ ]:
movies_df.columns

# Weighted Average Ratings...

In [ ]:
C=movies_df['vote_average'].mean()
C

In [ ]:
m=movies_df['vote_count'].quantile(0.7)
m

In [ ]:
movie_df1=movies_df[movies_df['vote_count']>=m]

In [ ]:
movie_df1.shape

In [ ]:
def weighted_rating(df,m=m,C=C):
    v=df['vote_count']
    R=df['vote_average']
    return ((v*R)+(m*C))/(v+m)

In [ ]:
movie_df2=movie_df1.copy()
movie_df2['weighted_rating']=weighted_rating(movie_df1)

In [ ]:
movie_df2.head()

In [ ]:
movie_df3=movie_df2[['original_title','vote_average','vote_count','weighted_rating']]
movie_df3.head()

In [ ]:
plot=movie_df3.sort_values('weighted_rating',ascending=False).head(10)
plot

In [ ]:
import matplotlib.pyplot as plt
bar=plt.bar(plot['original_title'],plot['weighted_rating'],color='red')
plt.xticks(rotation=90)
plt.title('Top 10 movies on weighted rating scale')
def autolabel(bars):
    for bar in bars:
        height=bar.get_height()
        plt.annotate('{height:.2f}'.format(height=height),xy=(bar.get_x(),height),xytext=(0,1),textcoords='offset points')
autolabel(bar)        
plt.show()

In [ ]:
popular=movie_df2[['original_title','popularity']]
plot1=popular.sort_values('popularity',ascending=False).head(10)
plot1

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.bar(plot1['original_title'],plot1['popularity'])
plt.xticks(rotation=90)
plt.xlabel('movie title')
plt.ylabel('popularity score')
plt.title('top 10 movies by popularity')
plt.show()

# Weighted Average Rating + Popularity

In [ ]:
mean=movie_df2['popularity'].mean()
std=movie_df2['popularity'].std()
movie_df2['norm_popularity']=(movie_df2['popularity']-mean)/std

In [ ]:
movie_df2.head()

In [ ]:
movie_df2['new_rating']=movie_df2['weighted_rating']+movie_df2['norm_popularity']
movie_df2.head()

In [ ]:
plot3=movie_df2[['original_title','new_rating']]
plot4=plot3.sort_values('new_rating',ascending=False).head(10)
plot4

# Recommendation

In [ ]:
plt.bar(plot4['original_title'],plot4['new_rating'])
plt.xticks(rotation=90)
plt.xlabel('movie title')
plt.ylabel('Rating')
plt.title('top 10 movies on weighted average and popularity')
plt.show()

# Content Based Filtering

In [ ]:
movies_df['overview'][0]

# Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english',analyzer='word')
movies_df['overview']=movies_df['overview'].fillna('')
tfidf_mat=tfidf.fit_transform(movies_df['overview'])
tfidf_mat.shape

In [ ]:
movies_df.shape

# Similarity Metrics

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
similarity=linear_kernel(tfidf_mat,tfidf_mat)

In [ ]:
similarity.shape

In [ ]:
indices=pd.Series(movies_df.index,index=movies_df['original_title'])

In [ ]:
def recommendation(title,sim=similarity):
    idx=indices[title]
    similar=list(enumerate(sim[idx]))
    similar=sorted(similar,key=lambda x:x[1],reverse=True)
    similar=similar[1:11]
    movie_idx=[i[0] for i in similar]
    return movie_idx,idx

# Recommendation on Content

In [ ]:
movie_index,idx=recommendation('Toy Story')
for i in movie_index:
    print(movies_df['original_title'].iloc[i],'-->',similarity[idx][i])